In [23]:
# !python --version
# %pip install --upgrade pandas
# %pip install openpyxl
# %pip install numpy
# %pip install scikit-learn
# %pip install matplotlib
# %pip install plot_keras_history
# %pip install tensorflow
# %pip install keras-tuner==1.0.1

In [24]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
        print("Setting GPU Memory Growth...")
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import time
import datetime

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from plot_keras_history import plot_history
import sklearn.metrics as metrics
import os as os

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.python.client import device_lib

import kerastuner as kt
from kerastuner.tuners import RandomSearch, BayesianOptimization


print("TF version:", tf.__version__)
print("KT version:", kt.__version__)
print("Panda version:", pd.__version__)

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print("Physical devices: ", tf.config.list_physical_devices())
# Se va a habilitar la dedicacion dinamica de memoria para que la GPU vaya asignando recursos al proceso conforme los vaya necesitando

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
#print(tf.config.list_physical_devices('GPU'))
print("Build with CUDA: ", tf.test.is_built_with_cuda())

# Number of measurements to predict
numPredictions = 20

# Number of best models
numBestModels = 1 

# Testing set percentage
test_size = 0.3
# Validation set percentage
val_size = 0.3


# Batch size
batch = 1024

# Nodos internos
hidden_nodes = 10

# Units parameters
minUnits = 50
maxUnits = 200
stepsUnits = 50
defaultUnits = 50

# Layers parameters
minLayers = 1
maxLayers = 10
defaultLayers = 3

# Dropout Parameters
minDropout = 0
maxDropout = 0.33
defaultDropout = .25

# Establecer medida de loss
# loss = "mean_squared_error"
loss = "mean_absolute_error"
# loss = "mean_absolute_percentage_error"

# Learning rate
learningRate = [0.0, 1e-2, 1e-3, 1e-4]

# Optimizer objetive: error percetange with the validation set 
objective = kt.Objective('val_mean_absolute_error', 'min')

# Maximum model trials and executions
trials = 5
executions = 5

# Model saving parameters
workingDirectory = datetime.datetime.fromtimestamp(time.time()).strftime('%d-%m-%Y-%H:%M')
# projectName = "lstm"



Setting GPU Memory Growth...


c:\Users\hecto\.conda\envs\myenv\lib\site-packages\tensorflow_core\python\client\session.py:1752: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


TF version: 2.1.0
KT version: 1.0.1
Panda version: 1.1.5
Physical devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Default GPU Device: /device:GPU:0
Num GPUs Available:  1
Build with CUDA:  True


In [25]:
# Epochs
epchs = 1000
projectName = "lstm30min2016-1kepochs-rs"
# projectName = "lstm30min2016-1kepochs-bo"


In [26]:
df = pd.read_excel("C:/Users/hecto/Documents/Master/TFM/tfm-renewable-energy-deep-learning/data/2016_30min.xlsx",
                    header = None, 
                    engine = 'openpyxl')[0]

df2 = pd.read_excel("C:/Users/hecto/Documents/Master/TFM/tfm-renewable-energy-deep-learning/data/2016_30min.xlsx",
                    header = None, 
                    engine = 'openpyxl')[0]



In [27]:
# Data preprocessing: each row will contain the 20 measures for each day , and the 10 measures for the following day
X = pd.DataFrame(np.array(df).reshape(-1, 20))
Y = pd.DataFrame.copy(X)

Y.columns = ["col_{}".format(i) for i in range(21, 41)]
Y = Y.drop(0)
Y = Y.reset_index(drop = True)
Y.loc[len(Y)] = np.zeros(numPredictions)

# Last row is deleted because it is the one used for the real prediction, 
# it is not useful for the training of the model. 
X.drop(X.tail(1).index, inplace = True)
Y.drop(Y.tail(1).index, inplace = True)

print("X Preproccessed shape: ", X.shape)
print("Y Preproccessed shape: ", Y.shape)
print("---------------------------------------------")

#  Uncomment in order to normalize data
# Data Normalization
# scaler = MinMaxScaler()
# X = pd.DataFrame(scaler.fit_transform(X), columns=["col_{}".format(i) for i in range(1, 21)])
# Y = pd.DataFrame(scaler.fit_transform(Y), columns=["col_{}".format(i) for i in range(21, 41)])

dfPreproccessed = pd.concat([X, Y], axis = 1)

print("DataFrame Preproccessed:")
print(dfPreproccessed)
print("---------------------------------------------")

# Split the data into training and validation sets
xTrain, xTest, yTrain, yTest = train_test_split(X, 
                                                Y, 
                                                test_size = test_size, 
                                                random_state = 0, 
                                                shuffle=False)
xTrain, xVal, yTrain, yVal = train_test_split(xTrain, 
                                            yTrain, 
                                            test_size = val_size, 
                                            random_state = 0,
                                            shuffle=False)

# Adding the third dimension needed to use LSTM (samples, timestamps, features)
xtrain = np.reshape(xTrain.values, (xTrain.shape[0], xTrain.shape[1], 1))
ytrain = np.reshape(yTrain.values, (yTrain.shape[0], yTrain.shape[1], 1))
xval = np.reshape(xVal.values, (xVal.shape[0], xVal.shape[1], 1))
yval = np.reshape(yVal.values, (yVal.shape[0], yVal.shape[1], 1))
xtest = np.reshape(xTest.values, (xTest.shape[0], xTest.shape[1], 1))
ytest = np.reshape(yTest.values, (yTest.shape[0], yTest.shape[1], 1))

# New shapes after third dimension added
print("Dimensions afther third dimension added:")
print("xTrain:\t"+str(xtrain.shape))
print("yTrain:\t"+str(ytrain.shape))
print("xVal:\t"+str(xval.shape))
print("yVal:\t"+str(yval.shape))
print("xTest:\t"+str(xtest.shape))
print("yTest:\t"+str(ytest.shape))

validation_data = xval.reshape((xval.shape[0], xval.shape[1]))

X Preproccessed shape:  (365, 20)
Y Preproccessed shape:  (365, 20)
---------------------------------------------
DataFrame Preproccessed:
              0           1           2           3           4           5  \
0    366.947667  481.168833  453.004333  548.509167  470.442333  504.718167   
1    361.308000  462.444333  543.777500  610.770167  667.571333  716.031167   
2    107.323333  153.843000  192.324000  192.461333  228.902000  353.019000   
3    142.967833  279.152667  315.124667  326.788500  350.171000  465.000833   
4     31.662500   62.964000   75.505000  126.278333  159.634333  220.034333   
..          ...         ...         ...         ...         ...         ...   
360  316.877033  231.862000  320.670567  444.582867  635.058133  613.942400   
361  329.211933  415.114133  448.318467  305.868667  336.630633  315.657133   
362  306.949633  344.887433  496.928467  454.899133  634.748100  589.407300   
363  344.823800  218.141167  268.108967  609.575467  549.478633  678.72

In [28]:
# Model definition:
with tf.device('/gpu:0'): 
    
    def build_model(hp): 

        model = Sequential()
        
        # model.add(LSTM(hidden_nodes, input_shape=(xtrain.shape[1], 1)))
        # First layer: LSTM. Number of units optimization
        model.add(LSTM(units = hp.Int('units', 
                                    min_value = minUnits,
                                    max_value = maxUnits, 
                                    step = stepsUnits,
                                    default = defaultUnits),
                        return_sequences = True, 
                        input_shape = (xtrain.shape[1], xtrain.shape[2])))

        # Second layer: Dropout with percetange optimization
        model.add(
            Dropout(hp.Float('dropout', 
                            min_value = minDropout,
                            max_value = maxDropout,
                            default = defaultDropout)))
        
        # Last layer:
        model.add(Dense(numPredictions))

        print("Summary: ")
        model.summary()

        # # Set Epsilon to 1, in order to fix huge MAPE values.
        # keras.backend.set_epsilon(1)

        # Model compilation: learning rate optimization
        model.compile(loss = loss,
                    optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', 
                                                                values = learningRate)),
                    metrics = [keras.metrics.MAE, 
                            keras.metrics.MAPE, 
                            keras.metrics.MSE])

        return model

        # history = model.fit(
        #     xtrain.reshape((xtrain.shape[0], xtrain.shape[1], 1)),
        #     ytrain,
        #     epochs = epchs,
        #     batch_size = batch,
        #     validation_data = (xval.reshape((xval.shape[0], xval.shape[1], 1)), yval))
    

In [29]:
# Model training:
with tf.device('/gpu:0'): 
    # Create optimizer using Random Search
    lstmOptimizer = RandomSearch(build_model, 
                                objective = objective,
                                max_trials = trials, 
                                executions_per_trial = executions,
                                project_name = projectName,
                                overwrite=True)
    
    # Create optimizer using Bayesian Optimization 
    # tcnOptimizer = BayesianOptimization(build_model, 
    #                             objective = objective,
    #                             max_trials = trials, 
    #                             executions_per_trial = executions,
    #                             project_name = projectName,
    #                             overwrite=True)

    # Search space summary
    lstmOptimizer.search_space_summary()

    # Search execution with epochs
    initialTime = time.time()
    lstmOptimizer.search(x = xtrain,
                        y = ytrain, 
                        epochs = epchs,
                        batch_size = batch,
                        validation_data = (xval.reshape((xval.shape[0], xval.shape[1], 1)), yval))
    finalTime = time.time()

    # Results summary
    lstmOptimizer.results_summary()

    # Get best model generated
    model = lstmOptimizer.get_best_models(num_models = numBestModels)[0]

    # Model training with epochs 
    history = model.fit(
                xtrain.reshape((xtrain.shape[0], xtrain.shape[1], 1)),
                ytrain,
                epochs = epchs,
                batch_size = batch,
                validation_data = (xval.reshape((xval.shape[0], xval.shape[1], 1)), yval))

    # Show model info
    print("Tiempo de entrenamiento (en segundos):\t"+str(finalTime - initialTime))
    print("Tiempo de entrenamiento (en horas):\t"+str((finalTime - initialTime)/3600))
    print(history.history.keys())
    print(model.history.history)
    print(model.history.params)


Summary: 
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 20, 50)            0         
_________________________________________________________________
dense (Dense)                (None, 20, 20)            1020      
Total params: 11,420
Trainable params: 11,420
Non-trainable params: 0
_________________________________________________________________


Summary: 
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 20, 50)            0         
_________________________________________________________________
dense (Dense)                (None, 20, 20)            1020      
Total params: 11,420
Trainable params: 11,420
Non-trainable params: 0
_________________________________________________________________
Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 1ms/sample


KeyboardInterrupt: 

In [ ]:
# Model evaluation:

# Training and Validation loss curves
plt.plot(history.history['loss'], label = 'Training loss')
plt.plot(history.history['val_loss'], label = 'Validation loss')
plt.legend()
plt.show()

# Model evaluation with validation data
score = model.evaluate(xtest.reshape((xtest.shape[0], xtest.shape[1], 1)), ytest)
print('Score:', score)